In [ ]:
!which qiime

In [ ]:
#import data to qiime2

!qiime tools import \
--type 'SampleData[PairedEndSequencesWithQuality]' \
--input-path for_qiime_uploadDRMB1.csv \
--output-path paired-end-demux.qza \
--source-format PairedEndFastqManifestPhred33

In [ ]:
!ls

In [ ]:
#demultiplex data
!qiime demux summarize \
--verbose \
--i-data paired-end-demux.qza \
--o-visualization paired-end-demux.qzv

In [ ]:
#trim PCR primers
!qiime cutadapt trim-paired \
--i-demultiplexed-sequences paired-end-demux.qza \
--p-cores 16 \
--p-front-f GTGYCAGCMGCCGCGGTAA \
--p-front-r GGACTACNVGGGTWTCTAAT \
--o-trimmed-sequences demux_trim.qza \
--verbose


In [ ]:
#summarize data
!qiime demux summarize \
--i-data demux_trim.qza \
--o-visualization demux_trim.qzv

In [ ]:
#run dada2 for ASV classification
!qiime dada2 denoise-paired \
--i-demultiplexed-seqs demux_trim.qza \
--p-trim-left-f 10 \
--p-trim-left-r 10 \
--p-trunc-len-f 200 \
--p-trunc-len-r 110 \
--p-n-threads 16  \
--o-representative-sequences rep_seqs_DRMB1.qza \
--o-table table_DRMB1.qza \
--o-denoising-stats dada_stats_DRMB1.qza

In [ ]:
%time!qiime feature-table summarize \
  --i-table table_DRMB1.qza \
  --o-visualization table_DRMB1.qzv\
  --m-sample-metadata-file DR_MB1_meta_seqID.txt

%time!qiime feature-table tabulate-seqs \
  --i-data rep_seqs_DRMB1.qza \
  --o-visualization rep_seqs_DRMB1.qzv

In [ ]:
#Assign taxonomy
%time!qiime feature-classifier classify-sklearn \
  --i-classifier silva-132-99-515-806-nb-classifier.qza\
  --i-reads rep_seqs_DRMB1.qza \
  --o-classification taxonomy__DRMB1.qza

In [ ]:
#Keep bacteria and archaea in ASV table
%time!qiime taxa filter-table \
  --i-table table_DRMB1.qza \
  --i-taxonomy taxonomy__DRMB1.qza \
  --p-exclude mitochondria,chloroplast \
  --p-include D_0__Bacteria,D_0__Archaea \
  --o-filtered-table table-BacArc_DRMB1.qza

In [ ]:
#Keep bacteria and archaea sequences
%time!qiime taxa filter-seqs \
  --i-sequences  rep_seqs_DRMB1.qza \
  --i-taxonomy taxonomy__DRMB1.qza \
  --p-exclude mitochondria,chloroplast \
  --p-include D_0__Bacteria,D_0__Archaea \
  --o-filtered-sequences rep_seqs_BacArc_DRMB1.qza

In [ ]:
!qiime taxa barplot \
  --i-table table-BacArc_DRMB1.qza \
  --i-taxonomy  taxonomy__DRMB1.qza \
  --m-metadata-file  DR_MB1_meta_seqID.txt\
  --o-visualization taxa-bar-plotsBacArc_DRMB1.qzv

In [ ]:
#align sequences for tree building 
%time!qiime alignment mafft \
  --i-sequences  rep_seqs_BacArc_DRMB1.qza \
  --o-alignment aligned-rep_seqs_BacArc_DRMB1.qza

In [ ]:
#mask sequences from alignment 
!qiime alignment mask \
  --i-alignment aligned-rep_seqs_BacArc_DRMB1.qza \
  --o-masked-alignment aligned-rep_seqsMasked_BacArc_DRMB1.qza

In [ ]:
#build phylogenetic tree 
!qiime phylogeny fasttree \
  --i-alignment aligned-rep_seqsMasked_BacArc_DRMB1.qza \
  --o-tree unrooted-tree-BacArc_DRMB1.qza

In [ ]:
#root tree
!qiime phylogeny midpoint-root \
  --i-tree unrooted-tree-BacArc_DRMB1.qza \
  --o-rooted-tree rooted-tree-BacArc_DRMB1.qza

In [ ]:

%time!qiime feature-table summarize \
  --i-table table-BacArc_DRMB1.qza \
  --o-visualization table-BacArc_DRMB1.qzv\
  --m-sample-metadata-file DR_MB1_meta_seqID.txt

%time!qiime feature-table tabulate-seqs \
  --i-data rep_seqs_BacArc_DRMB1.qza \
  --o-visualization rep_seqs_BacArc_DRMB1.qzv

In [ ]:
#export data from qiime2
!qiime tools export \
    table-BacArc_DRMB1.qza \
  --output-dir exported_DRMB1/

!qiime tools export \
 rooted-tree-BacArc_DRMB1.qza \
 --output-dir exported_DRMB1/

!qiime tools export \
  taxonomy__DRMB1.qza \
  --output-dir exported_DRMB1/


In [ ]:
#Remove mito and chloroplast from taxonomy, remove header and sort to
#relabel on R
!wc -l taxonomy.tsv

!grep 'Bacteria\|Archaea' exported_DRMB1/taxonomy.tsv | wc -l 
!grep 'Bacteria\|Archaea' exported_DRMB1/taxonomy.tsv | head

!grep  'Bacteria\|Archaea' exported_DRMB1/taxonomy.tsv | \
tail -n +2 |  \
sort -k1,1 | wc -l

!grep  'Bacteria\|Archaea' exported_DRMB1/taxonomy.tsv | \
tail -n +2 |  \
sort -k1,1   > exported_DRMB1/tax-no-BacArc_DRMB1.txt




In [ ]:
!biom  convert \
-i /exported_DRMB1/feature-table.biom \
--to-tsv \
-o /space/home/rosales/projects/2018_micro_resistance_cora/exported_DRMB1/feature-tableBacArc_DRMB1.txt

!wc -l /space/home/rosales/projects/2018_micro_resistance_cora/exported_DRMB1/feature-tableBacArc_DRMB1.txt

In [ ]:
mv /space/home/rosales/projects/2018_micro_resistance_cora/exported_DRMB1/tree.nwk /space/home/rosales/projects/2018_micro_resistance_cora/exported_DRMB1/tree_BacArc_DRMB1.nwk